## 기계변역 예제
- 순환층을 사용하는 시퀀스 모델 제작 ~ 완전한 트랜스포머 아키텍처 구현
### 데이터 셋 로딩

In [ ]:
## 데이터 셋 로딩
# https://www.manythings.org/anki의 영어 - 프랑스 번역 셋 이용
!wget http://storage.googleeapis.com/download.tensorflow.org/data/fra-eng.zip
!unzip -q fra-eng.zip

In [ ]:
# 데이터 살펴보기
text = 'fra-eng/fra.txt'
with open(text) as f:
    lines = f.read().split('\n')[:-1]
text_pairs = []
for line in lines: # 라인별 처리
    eng, fra = line.split('\t')
    fra = '[start]' + fra + '[end]'
    text_pairs.append((eng, fra))
    
# 랜덤 문장보기    
import random
print(random.choice(text_pairs))

### 데이터 세트 분리
- 훈련, 검증, 테스트

In [ ]:
# 셔플후 세트 분리
random.shuffle(text_pairs)
num_val_sam = int(0.15 * len(text_pairs))
num_train_sam = len(text_pairs) - 2 * num_val_sam
train_pairs = text_pairs[: num_train_sam]
val_pairs = text_pairs[num_train_sam : num_train_sam + num_val_sam]
test_pairs = text_pairs[num_train_sam + num_val_sam : ]

### TV층 (TextVectorization)준비
- 영어층, 프랑스어층
- 문자열 전처리 방식 커스텀 (fra_standard)

In [ ]:
# 라이브러리 
import tensorflow as tf
import string
import re

# 프랑스어 TV층에 적용하기 위해 특수 문자들 삭제
str_chars = string.punctuation 
str_chars = str_chars.replace('[', '')
str_chars = str_chars.replace(']', '')
# 문자열 표준화 함수 정의
def fra_standard(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
    lowercase, f'[{re.escape(str_chars)}]', '')

# 범위제한을 위한 작업(예시라서 제한함)
vocab_size = 15000,
sequence_length = 20

#영어층
source_vec = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length,
)
# french layer
target_vec = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length + 1,
    standardize = fra_standard
)

# 훈련후 어휘 사전 만들기
train_eng_texts = [pair[0] for pair in train_pairs]
train_fra_texts = [pair[1] for pair in train_pairs]
source_vec.adapt(train_eng_texts)
target_vec.adapt(train_fra_texts)

### 번역작업을 위한 데이터셋 준비

In [ ]:
batch_size = 64

def formet_dataset(eng, fra):
    eng = source_vec(eng)
    fra = target_vec(fra)
    return ({ # 이 dict가 inputs
        'english': eng,
        'french' : fra[:,:-1],
    }, fra[:, 1:]) # fra가 target

def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return dataset.shuffle(2048).prefetch(16).cache() # 전처리 속도리를 높이기 위한 캐싱

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
# 데이터 셋 크기 확인
for inputs, targets in train_ds.take(1):
    print(f'inputs['eng'].shape : {inputs['eng'].shape}')
    print(f'inputs['fra'].shape : {inputs['fra'].shape}')
    print(f'targets.shape : {targets.shape}')

### RNN 을 사용한 StoS모델
- 시퀀스_투_시퀀스 모델